In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\abhis\\Desktop\\MLProjects\\Movie Recommender'

In [2]:
# Updating the entity (return type of a function) like configbox, and the example below
# same return from config.yaml
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [3]:
from MovieRecommender.constants import *
from MovieRecommender.utils.common import read_yaml, create_directories

In [4]:
# update congiuration manager cat configuration.py

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
    
        return data_ingestion_config

In [5]:
## Update components

import os
import urllib.request as request
import zipfile
from MovieRecommender.logging import logger
from MovieRecommender.utils.common import get_size

In [6]:
logger.info(f"hello")

[2023-06-13 03:06:29,674: INFO: 3427745012: hello]


In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """Downloads the file and save into local directory
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """Extracts zipfile into data directory
            Returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
## Create the pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-13 03:06:33,211: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-13 03:06:33,213: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-13 03:06:33,215: INFO: common: created directory at: artifacts]
[2023-06-13 03:06:33,219: INFO: common: created directory at: artifacts/data_ingestion]
[2023-06-13 03:07:32,960: INFO: 243203096: artifacts/data_ingestion/data.zip downloaded with following info: 
Date: Tue, 13 Jun 2023 08:06:33 GMT
Server: Apache/2.4.6 (Red Hat Enterprise Linux) OpenSSL/1.0.2k-fips
Last-Modified: Tue, 03 Dec 2019 17:15:37 GMT
ETag: "10846a59-598cfd8426696"
Accept-Ranges: bytes
Content-Length: 277113433
Connection: close
Content-Type: application/zip

]
